In [126]:
import numpy as np
import unicodedata
import re

## Text Normalization

##### Reading the training corpus

In [361]:
# Reading train.en txt file
with open('data/train.en.txt', encoding='utf8') as f:
    lines = f.readlines()
    
with open('data/train.nl.txt', encoding='utf8') as f:
    train_nl = f.readlines()
    
with open('data/train.xh.txt', encoding='utf8') as f:
    train_xh = f.readlines()
    
with open('data/train.zu.txt', encoding='utf8') as f:
    train_zu = f.readlines()
    
with open('data/train.af.txt', encoding='utf8') as f:
    train_af = f.readlines()

##### Text Normalization

In [362]:
lines = [line.lower() for line in lines]
train_nl = [line.lower() for line in train_nl]
train_xh = [line.lower() for line in train_xh]
train_zu = [line.lower() for line in train_zu]
train_af = [line.lower() for line in train_af]

In [363]:
train_zu = train_zu[49:]

In [364]:
new_lines = []
for i in range(len(lines)):
    new_lines += lines[i].split('. ')
        
train_nl_normalized = []
for i in range(len(train_nl)):
    train_nl_normalized += train_nl[i].split('. ')
    
train_xh_normalized = []
for i in range(len(train_xh)):
    train_xh_normalized += train_xh[i].split('. ')
    
train_zu_normalized = []
for i in range(len(train_zu)):
    train_zu_normalized += train_zu[i].split('. ')
    
train_af_normalized = []
for i in range(len(train_zu)):
    train_af_normalized += train_zu[i].split('. ')

In [365]:
train_zu[40]

'zikhona izinhlangano zepolitiki ezahlanganyela nokhongolose ezifana ne-sacp (amakomanisi), i-pac kanye nezinye ezifana no-bcm. kwahamba kanjalo eningizimu afrika kwaze kwaboshwa osopolitiki ngaleso sikhathi sengcindezelo abafana no nelson mandela, walter sisulu, govan mbeki, raymond mhlaba, robert sobukwe, andrew mlangeni, dennis goldberg, kasrils nabanye. kanti abanye baya ekudingisweni abafana no-oliver tambo, thabo mbeki, chris hani, duma nokwe, moses kotane. kwaze kwafika lapho umbuso wengcindezelo wehluleka khona uthathwa ngabamnyama. ngemva konyaka ka 1994, kwangena intando yeningi ("democracy"), lapho umongameli wokuqala wentando yeningi kwaba ngunelson mandela owaboshwa iminyaka ewu-27 ejele. ngemva kwalokho iningizimu afrika yaqala ukuvula uhwebo ihwebelana namanye amazwe omhlaba. iningizimu afrika yabumba umthethosisekelo oqukethe intando yeningi amalungelo wobuntu, nobulungiswa obungakhethi ibala. kanti zonke izinhlanga zaqala ukuba nelungelo lokuvota ngokukhululeka. kanti 

In [366]:
for i in range(len(new_lines)):
    new_lines[i] = new_lines[i].replace('.', '')
    new_lines[i] = new_lines[i].replace('\n', '')
    new_lines[i] = new_lines[i].replace('-', ' ') # replace hiphens with spaces
    #new_lines[i] = re.sub('ī', 'i', new_lines[i])
    #new_lines[i] = re.sub('[āäå]', 'a', new_lines[i])
    #new_lines[i] = re.sub('ū', 'u', new_lines[i]) 
    #new_lines[i] = re.sub('[éēè]', 'e', new_lines[i])
    #new_lines[i] = re.sub('[ö]', 'o', new_lines[i])
    new_lines[i] = unicodedata.normalize('NFD', new_lines[i])
    new_lines[i] = re.sub('\d', '0', new_lines[i])
    new_lines[i] = re.sub('[^a-z0 ]', '', new_lines[i])

In [367]:
for n in new_lines:
    if 'jag' in n:  # check if the special charachters in Nä jag har smörgåsar disappeared
        print(n)

for example a swedish programmer mailing another programmer asking if they should go for lunch could get n jag har smrgsar as the answer which should be na jag har smorgasar meaning no ive got sandwiches


In [368]:
# Add start and end tokens
for i in range(len(new_lines)):
    new_lines[i] = "((" + new_lines[i] + ")"

In [369]:
new_lines[17]

'((the suffix  ism denotes the ideological current that favours anarchy)'

In [338]:
def normalize_text(corpus):
    for i in range(len(corpus)):
        corpus[i] = corpus[i].replace('.', '')
        corpus[i] = corpus[i].replace('\n', '')
        corpus[i] = corpus[i].replace('-', ' ') # replace hiphens with spaces
        corpus[i] = unicodedata.normalize('NFD', corpus[i])
        corpus[i] = re.sub('\d', '0', corpus[i])
        corpus[i] = re.sub('[^a-z0 ]', '', corpus[i])
        
        corpus[i] = "((" + corpus[i] + ")"

In [370]:
normalize_text(train_zu_normalized)
train_zu_normalized[40]

'((ukubhalwa kolimi lwesizulu kwakwenganyelwe yizulu language board yakwazulu natal)'

In [371]:
normalize_text(train_xh_normalized)
normalize_text(train_nl_normalized)
normalize_text(train_af_normalized)

## Language Modelling

In [372]:
trigrams = dict()
for c1 in 'abcdefghijklmnopqrstuvwxyz 0(':
    for c2 in 'abcdefghijklmnopqrstuvwxyz 0()':
        if c2 == ")":
            continue
        if c2 == "(" and c1 != "(":
            continue
        for c3 in 'abcdefghijklmnopqrstuvwxyz 0)':
            trigram = c1+c2+c3
            trigrams[trigram] = trigrams.setdefault(trigram, 0)
            
for i in range(len(new_lines)):
    sentence = new_lines[i]
    for j in range(2, len(sentence)):
        trigram = sentence[j-2 : j+1]
        trigrams[trigram] += 1

In [373]:
trigrams['the']

10112

In [374]:
def create_trigrams(train_lang):
    lang_trigrams = dict()
    for c1 in 'abcdefghijklmnopqrstuvwxyz 0(':
        for c2 in 'abcdefghijklmnopqrstuvwxyz 0()':
            if c2 == ")":
                continue
            if c2 == "(" and c1 != "(":
                continue
            for c3 in 'abcdefghijklmnopqrstuvwxyz 0)':
                trigram = c1+c2+c3
                lang_trigrams[trigram] = lang_trigrams.setdefault(trigram, 0)
            
    for i in range(len(train_lang)):
        sentence = train_lang[i]
        for j in range(2, len(sentence)):
            trigram = sentence[j-2 : j+1]
            lang_trigrams[trigram] += 1
            
    return lang_trigrams

In [405]:
nl_trigrams = create_trigrams(train_nl_normalized)
nl_trigrams["die"]

942

In [406]:
zu_trigrams = create_trigrams(train_zu_normalized)
xh_trigrams = create_trigrams(train_xh_normalized)
af_trigrams = create_trigrams(train_af_normalized)

In [407]:
af_trigrams["the"]

442

#### Smoothing

In [218]:
# Add alpha
alpha = 0.1
for k, v in trigrams.items():
    trigrams[k] += alpha

In [219]:
# create a dict of all w_t-2 and w_t-1
trigrams_first_2letters = dict()

for k, v in trigrams.items():
    key = k[:2]
    trigrams_first_2letters[key] = trigrams_first_2letters.setdefault(key, 0) + v

In [220]:
# Normalize the dicts
for k, v in trigrams.items():
    trigrams[k] = v / trigrams_first_2letters[k[:2]]

In [221]:
sum_aa = 0
for k, v in trigrams.items():
    if k[:2] == '((':
        sum_aa += v
        
sum_aa

0.9999999999999998

In [409]:
def smoothing(alpha, lang_trigrams):
    for k, v in lang_trigrams.items():
        lang_trigrams[k] += alpha
    
    first_2letters = dict()
    for k, v in lang_trigrams.items():
        key = k[:2]
        first_2letters[key] = first_2letters.setdefault(key, 0) + v
        
    for k, v in lang_trigrams.items():
        lang_trigrams[k] = v / first_2letters[k[:2]]

In [410]:
smoothing(0.1, af_trigrams)
smoothing(0.1, nl_trigrams)
smoothing(0.1, zu_trigrams)
smoothing(0.1, xh_trigrams)

#### Generation

In [222]:
en_sentence = ""
start_tokens = []
start_tok_props = []
#for k, v in trigrams.items():
    # start sentence
    #if k[:2] == "((":
        #start_tokens.append()
        
start_sentence = {k:v for k, v in trigrams.items() if "((" == k[:2]}
en_sentence += np.random.choice(list(start_sentence.keys()), p = list(start_sentence.values()))

while en_sentence[-1] != ")":
    trigram_prev_tokens = {k:v for k, v in trigrams.items() if en_sentence[-2:] == k[:2]}
    en_sentence += np.random.choice(list(trigram_prev_tokens.keys()), p = list(trigram_prev_tokens.values()))[-1]
    
en_sentence

'((by se wider therbegoin frovers suce)'

In [413]:
nl_sentence = ""
        
start_sentence = {k:v for k, v in nl_trigrams.items() if "((" == k[:2]}
nl_sentence += np.random.choice(list(start_sentence.keys()), p = list(start_sentence.values()))

while nl_sentence[-1] != ")":
    trigram_prev_tokens = {k:v for k, v in nl_trigrams.items() if nl_sentence[-2:] == k[:2]}
    nl_sentence += np.random.choice(list(trigram_prev_tokens.keys()), p = list(trigram_prev_tokens.values()))[-1]
    
nl_sentence

'((arver naar alde an vordveras corrifierzalijfdepsyriveran mandatel mot conallij aarvert adiuste one)'

In [422]:
def generate(lang_trigrams):
    lang_sentence = ""
    
    start_sentence = {k:v for k, v in lang_trigrams.items() if "((" == k[:2]}
    lang_sentence += np.random.choice(list(start_sentence.keys()), p = list(start_sentence.values()))
    
    while lang_sentence[-1] != ")":
        trigram_prev_tokens = {k:v for k, v in lang_trigrams.items() if lang_sentence[-2:] == k[:2]}
        lang_sentence += np.random.choice(list(trigram_prev_tokens.keys()), p = list(trigram_prev_tokens.values()))[-1]
        
    return lang_sentence

In [427]:
generate(zu_trigrams)

'((0000 000 ba stana aphazumi afundazweni)'

In [428]:
generate(xh_trigrams)

'((umfuni ekane kozis t fungombisebe ukuphazintin wakangamasela imits celandlantla zokuca ekund tabanayo 00 ko ooli omhlamnyamaphetheleliswayaka iimpithi zekusogokulendon bekantlacadisiboseni)'

In [429]:
generate(af_trigrams)

'(( lithetshambuye amu kati)'

#### Text Generation on Validation set

In [414]:
val_lines = ""
with open('data/val.en.txt', encoding='utf8') as f:
    val_lines = f.readlines()
    
with open('data/val.nl.txt', encoding='utf8') as f:
    nl_val = f.readlines()
    
with open('data/val.af.txt', encoding='utf8') as f:
    af_val = f.readlines()
    
with open('data/val.zu.txt', encoding='utf8') as f:
    zu_val = f.readlines()
    
with open('data/val.xh.txt', encoding='utf8') as f:
    xh_val = f.readlines()

In [276]:
for i in range(len(val_lines)):
    val_lines[i] = val_lines[i].replace(' \n', ')')
    val_lines[i] = val_lines[i].replace('\n', ')')
    val_lines[i] = "((" + val_lines[i]

In [277]:
val_lines[918]

'((what is grade of service and how is it measured)'

In [279]:
count_props = 0
log_prop_sum = 0
for i in range(len(val_lines)):
    sentence = val_lines[i]
    for c in range(len(sentence)-2):
        #print(sentence[c:c+3])
        count_props += 1
        log_prop_sum += np.log2(trigrams[sentence[c:c+3]])

In [285]:
print("The perplexity is:", 2**((-1/count_props)*log_prop_sum))

The perplexity is: 7.8131404061607785


In [431]:
def add_tokens(lang_val):
    for i in range(len(lang_val)):
        lang_val[i] = lang_val[i].replace(' \n', ')')
        lang_val[i] = lang_val[i].replace('\n', ')')
        lang_val[i] = "((" + lang_val[i]
        
add_tokens(nl_val)
add_tokens(zu_val)
add_tokens(af_val)
add_tokens(xh_val)

In [433]:
def calc_perplexity(lang_val, lang_trigrams):
    count_props = 0
    log_prop_sum = 0
    for i in range(len(lang_val)):
        sentence = lang_val[i]
        for c in range(len(sentence)-2):
            count_props += 1
            log_prop_sum += np.log2(lang_trigrams[sentence[c:c+3]])
            
    return 2**((-1/count_props)*log_prop_sum)

In [436]:
print("The perplexity of the Dutch validation corpus is:", calc_perplexity(nl_val, nl_trigrams))
print("The perplexity of the Zulu validation corpus is:", calc_perplexity(zu_val, zu_trigrams))
print("The perplexity of the Xhosa validation corpus is:", calc_perplexity(xh_val, xh_trigrams))
print("The perplexity of the Afrikaans validation corpus is:", calc_perplexity(af_val, af_trigrams))

The perplexity of the Dutch validation corpus is: 7.881499297115041
The perplexity of the Zulu validation corpus is: 8.494299341750839
The perplexity of the Xhosa validation corpus is: 8.341888473261541
The perplexity of the Afrikaans validation corpus is: 28.990054195692462


## Language Identification

In [437]:
languages = ["af", "en", "nl", "xh", "zu"]
with open('data/test.lid.txt', encoding='utf8') as f:
    test = f.readlines()

In [438]:
test

['zu encwadini kamormoni ithi ngokuba bheka iningi lamadodakazi amalamani bathathe iziboshwa futhi babaphuca lokho obekuthandwa kakhulu nokuyigugu ngaphezu kwazo zonke izinto okuwubumsulwa futhi ubuhle enkulumweni yengqungquthela urichard g \n',
 'zu umbhalo \n',
 'af op daardie stadium het die n 0 deur die grootrivierpas gegaan \n',
 'nl zuidafrika \n',
 'af n 0 nasionale pad \n',
 'xh waaselemanukuzingela umbuyeleli ngokwenyamakazi ezama ukumtshabalalisa \n',
 'zu ukushaya indlwabu akuvumelekile kodwa akubhekwa njengobutabane \n',
 'xh umzi okokwawo ubukholelwe kulibe ngeendawana ezithile nangenxa yokuba ezelwe wayinkulu \n',
 'en the theremin an electronic musical instrument traditionally uses the heterodyne principle to produce a variable audio frequency in response to the movement of the musicians hands in the vicinity of one or more antennas which act as capacitor plates \n',
 'zu imisebenzi ethathwa njengokuphula umthetho wobumsulwa ifaka phakathi kokubili ukuphinga nobufebe kan